### Компания хочет понять, насколько равномерно курьеры работают в течение месяца. Для этого нужно найти ID курьера с наибольшей разницей между максимальной и минимальной средней дневной скоростью в июне 2021 года.

In [114]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import month, year, col, round, avg, min, max, desc

In [115]:
spark = SparkSession.builder \
    .appName("BiggestDiffBetweenSpeeds") \
    .master("local[*]") \
    .config("spark.driver.extraJavaOptions", "-Djava.security.manager=allow") \
    .getOrCreate()

#### Step 1. Loading and filtration data

In [116]:
try:
    df = spark.read.parquet("data/couriers_orders.parquet")
    df.show(7)
except FileNotFoundError as e:
    print(f"{e}")
    spark.stop()

+-------------------+----------+--------+--------+-----------+
|               date|courier_id|order_id|distance|travel_time|
+-------------------+----------+--------+--------+-----------+
|2021-07-12 00:00:00|        10|       1|     1.9|      36.17|
|2021-07-02 00:00:00|         3|       2|    3.98|      21.34|
|2021-04-15 00:00:00|         6|       3|    3.98|      43.33|
|2021-07-16 00:00:00|        10|       4|    2.85|      14.01|
|2021-06-11 00:00:00|        10|       5|    4.89|      32.09|
|2021-04-21 00:00:00|         9|       6|    1.06|      18.17|
|2021-07-12 00:00:00|         1|       7|    0.58|      19.22|
+-------------------+----------+--------+--------+-----------+
only showing top 7 rows


In [117]:
june_df = df.filter(
    (month(col("date")) == 6) &
    (year(col("date")) == 2021)
)

june_df = june_df.filter(
    (col("distance") > 0) &
    (col("travel_time") > 0)
)

june_df.show(5)

+-------------------+----------+--------+--------+-----------+
|               date|courier_id|order_id|distance|travel_time|
+-------------------+----------+--------+--------+-----------+
|2021-06-11 00:00:00|        10|       5|    4.89|      32.09|
|2021-06-14 00:00:00|         4|       9|    4.13|      29.34|
|2021-06-27 00:00:00|         8|      10|    1.04|      12.56|
|2021-06-27 00:00:00|         1|      19|    1.85|      13.56|
|2021-06-28 00:00:00|         2|      25|    4.02|      12.43|
+-------------------+----------+--------+--------+-----------+
only showing top 5 rows


#### Step 2. New column "speed_km/h"

In [118]:
june_df = june_df.withColumn(
    "speed_km/h",
    col("distance") / (col("travel_time") / 60)
)

june_df.show(5)

+-------------------+----------+--------+--------+-----------+------------------+
|               date|courier_id|order_id|distance|travel_time|        speed_km/h|
+-------------------+----------+--------+--------+-----------+------------------+
|2021-06-11 00:00:00|        10|       5|    4.89|      32.09| 9.143035213462136|
|2021-06-14 00:00:00|         4|       9|    4.13|      29.34| 8.445807770961146|
|2021-06-27 00:00:00|         8|      10|    1.04|      12.56| 4.968152866242038|
|2021-06-27 00:00:00|         1|      19|    1.85|      13.56| 8.185840707964601|
|2021-06-28 00:00:00|         2|      25|    4.02|      12.43|19.404666130329844|
+-------------------+----------+--------+--------+-----------+------------------+
only showing top 5 rows


#### Step 3. Average daily speed for a courier

In [119]:
daily_avg_speed_df = june_df \
    .groupBy("courier_id", "date") \
    .agg(avg("speed_km/h").alias("daily_avg_speed_km/h"))

daily_avg_speed_df.show(5)

+----------+-------------------+--------------------+
|courier_id|               date|daily_avg_speed_km/h|
+----------+-------------------+--------------------+
|         8|2021-06-08 00:00:00|   7.203231684050229|
|         8|2021-06-22 00:00:00|   8.815838459709019|
|         1|2021-06-17 00:00:00|  11.136363636363637|
|         9|2021-06-01 00:00:00|    5.28627015008338|
|         2|2021-06-02 00:00:00|   4.196934868159241|
+----------+-------------------+--------------------+
only showing top 5 rows


#### Step 4. Min-Max daily speed for a courier

In [120]:
daily_min_max_speed_df = daily_avg_speed_df \
    .groupby("courier_id") \
    .agg(min("daily_avg_speed_km/h").alias("daily_min_speed"),
         max("daily_avg_speed_km/h").alias("daily_max_speed"),
         )

daily_min_max_speed_df.show(5)

+----------+------------------+------------------+
|courier_id|   daily_min_speed|   daily_max_speed|
+----------+------------------+------------------+
|         7|1.0535557506584723| 11.33508936970837|
|         6| 1.403067638923812| 25.49718574108818|
|         9|0.7547857793983591|13.315508021390373|
|         5| 1.104111823559212|17.651376146788987|
|         1|1.6655313351498635|25.342333654773388|
+----------+------------------+------------------+
only showing top 5 rows


#### Step 5. New column "speed diff"

In [121]:
daily_min_max_speed_df = daily_min_max_speed_df.withColumn(
    "speed_diff",
    col("daily_max_speed") - col("daily_min_speed")
)

daily_min_max_speed_df.show()

+----------+------------------+------------------+------------------+
|courier_id|   daily_min_speed|   daily_max_speed|        speed_diff|
+----------+------------------+------------------+------------------+
|         7|1.0535557506584723| 11.33508936970837|10.281533619049899|
|         6| 1.403067638923812| 25.49718574108818| 24.09411810216437|
|         9|0.7547857793983591|13.315508021390373|12.560722241992014|
|         5| 1.104111823559212|17.651376146788987|16.547264323229776|
|         1|1.6655313351498635|25.342333654773388|23.676802319623523|
|        10|1.7512012813667912| 18.60500379075057|16.853802509383776|
|         3| 1.253071253071253|13.351278600269179|12.098207347197926|
|         8|0.7925696594427245|14.946159368269923|14.153589708827198|
|         2|1.5167888846005404|19.404666130329844|17.887877245729303|
|         4|0.9739600946905647|25.776566757493192|24.802606662802628|
+----------+------------------+------------------+------------------+



#### Step 6. Courier with max diff

In [122]:
courier_min_max_diff_sorted_df = daily_min_max_speed_df.orderBy(desc("speed_diff"))

courier_min_max_diff_sorted_df.show()

top_courier = courier_min_max_diff_sorted_df.first()

+----------+------------------+------------------+------------------+
|courier_id|   daily_min_speed|   daily_max_speed|        speed_diff|
+----------+------------------+------------------+------------------+
|         4|0.9739600946905647|25.776566757493192|24.802606662802628|
|         6| 1.403067638923812| 25.49718574108818| 24.09411810216437|
|         1|1.6655313351498635|25.342333654773388|23.676802319623523|
|         2|1.5167888846005404|19.404666130329844|17.887877245729303|
|        10|1.7512012813667912| 18.60500379075057|16.853802509383776|
|         5| 1.104111823559212|17.651376146788987|16.547264323229776|
|         8|0.7925696594427245|14.946159368269923|14.153589708827198|
|         9|0.7547857793983591|13.315508021390373|12.560722241992014|
|         3| 1.253071253071253|13.351278600269179|12.098207347197926|
|         7|1.0535557506584723| 11.33508936970837|10.281533619049899|
+----------+------------------+------------------+------------------+



#### Step 7. Results

In [123]:
print("=" * 40)
print(f"    Courier with id: {top_courier["courier_id"]}")
print(f"    Maximum difference: {top_courier["speed_diff"]:.2f} km/h")
print(f"    Max: {top_courier["daily_max_speed"]:.2f} km/h, Min: {top_courier["daily_min_speed"]:.2f} km/h")
print("=" * 40)

spark.stop()

    Courier with id: 4
    Maximum difference: 24.80 km/h
    Max: 25.78 km/h, Min: 0.97 km/h
